In [ ]:
from gboml import GbomlGraph

gboml_model = GbomlGraph(1*24*365)
nodes, edges, _ = gboml_model.import_all_nodes_and_edges("../models/merge_components.txt")
gboml_model.add_nodes_in_model(*nodes)
gboml_model.add_hyperedges_in_model(*edges)
gboml_model.build_model()
solution = gboml_model.solve_gurobi(details=True)
(solution_flat, objective, status, solver_info, constr_info, var_info) = solution
solution_dict = gboml_model.turn_solution_to_dictionary(solution=solution_flat, solver_data=solver_info, status=status, objective=objective, constraint_info=constr_info, variables_info=var_info)
print(solution)

In [ ]:
import sys
sys.path.append("../src/")

In [ ]:
import analysis
analysis.show_results(solution_dict)

In [ ]:
import pandas as pd
results_week = pd.DataFrame()
elements = solution_dict["solution"]["elements"]
for element in elements.keys():
    if element in ["WIND_PLANT", "SOLAR_PV", "DEMAND_EL", "DISTRIBUTION_EL", "BATTERY"]:
        for variable in elements[element]["variables"].keys():
            var_values = elements[element]["variables"][variable]["values"]
            if len(var_values) == 1:
                result = f"Node {element}: {variable}: {var_values[0]}"
                print(result)
            else:
                col = variable
                results_week[col] = var_values[: 24*7]
                print(f"Node {element}: {variable}: (first week)")
                #plt.plot(var_values[: 24 * 7])
                #plt.show()

In [ ]:
results_week.head()

In [ ]:
results = results_week[96:120].reset_index(drop=True)
results.head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(24)
plt.rcParams["figure.figsize"] = 9, 6

plt.plot(x, results.consumption_el)
plt.plot(x, results.electricity_import)
plt.plot(x, results.wind_generated)
plt.plot(x, results.solar_generated)
plt.xlabel("Hour")
plt.ylabel("kW")
plt.legend(['demand', 'electricity_import', 'wind_generated', 'solar_generated'], loc='upper left')
plt.show()


In [ ]:
plt.rcParams["figure.figsize"] = 9, 6

plt.plot(x, results.consumption_el)
plt.plot(x, results.electricity_import)
plt.plot(x, results.solar_generated+results.wind_generated)
plt.xlabel("Hour")
plt.ylabel("kW")
plt.legend(['demand', 'electricity_import', 'solar + wind'], loc='upper left')
plt.show()


In [ ]:
results["operating_cost_electricity"]

In [ ]:
ax = results[[ 'wind_generated', 'solar_generated', 'electricity_import']].plot(kind='bar', stacked=True, figsize=(9, 6), rot=0, xlabel='Hour', ylabel='kW')
ax2 = results.operating_cost_electricity.plot(secondary_y=True,color="grey", ax=ax, label="operating cost electricity")
_ = ax.plot(results["consumption_el"],  color='black',  label='Demand')
_ = ax.legend()
_ = ax2.legend(loc="upper right")

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(13, 6))
_ = axes[0].plot(x, results["soc"], label="soc")
_ = axes[1].plot(x, results["discharge"], color="grey", label="discharge")
_ = fig.legend()